# Задание

In [9]:
import numpy as np

Сравните метрики hit_rate@k, precision@k.  
Какую метрику использовать предпочтительно и почему?  

### Ответ
**hit_rate@k** - менее информативна, т.к. отмечает лишь факт наличия успешной рекомендации. Может использоваться когда ожидается только одна покупка.

**precision@k** - более информативна, т.к. показывает степень успешности предложенных рекомендаций    
(в процентном соотношении сколько товаров из числа предложенных было куплено)

Предполагаю, что precision@k предпочтительнее, т.к. содержит в себе как степень успешности, так и факт наличия успешной рекомендации

Приведите пример 2-3 задач (опишите, что является клиентом, что товаром), в которой более уместно использовать метрику hit_rate?  

1) Подбор цвета конкретной модели шапки по фотографии человека в онлайн магазине:  
**П** - фотография  
**Т** - шапки разных цветов  

2) Подбор цвета iphone 12 по профилю и истории покупок магазина:  
**П** - клиент  
**Т** - устройства разных цветов

Похоже что hit_rate (отражающий факт покупки хотя бы одного из рекомендованных товаров) целесообразно применять, когда нужно сделать рекомендацию в рамках одного товара или группы тесно связанных, но взаимно исключающих товаров (вряд ли покупателю нужны сразу 2 похожие шапки разных цветов)

## Задание

В метрике NDCG@k мы используем логарифм в знаменателе.  
Как Вы думаете, почему именно логарифм?  
Какую функцию можно использовать вместо логарифма?  
Приведите пример метрик/подходов к предобработке данных/функций ошибок в ML, где также в знаменателе присутствует логарифм.  
**Precision, Recall, F-score, R2**

Попробую рассуждать, исходя из того, что я знаю о логарифме:

1) logN растет медленее N => значит метрика 
$$DCG = \frac{1}{|r|} \sum_u{\frac{[bought fact]}{discount(i)}}$$

будет расти медленнее, чем например
$$AP@k = \frac{1}{K} \sum{[recommended_{relevant_i}] * precision@k}$$

следовательно штраф за неправильное предсказание в NDCG будет меньше, чем в AP

2) вместо логарифма можно, видимо, использовать другую функцию, которая растет медленнее, чем $N$, например $\sqrt{N}$

ОДЗ у подлогарифменного выражения и подкоренного выражения у корня квадратного одинаковые - $f(x) > 0$

3) логарифмирование применялось в нашем курсе для нормализации, когда разброс значений отличается на порядки. И, кажется в kNN, чтобы разница между классами была более явной. Ну и в LogLoss. Чаще всего логарифм в знаменателе применяется для сглаживания

## Задание
 
Какие еще метрики (Вы можете вспомнить уже пройденные Вами или посмотреть в интернете) могут использоваться для рекомендательных систем (приведите примеры метрики и чем являются интеракции, чтобы она могла быть использована).  

ROC AUC - Насколько высока концентрация интересных товаров в начале списка рекомендаций (может применяться там же, где и precision)

pfound - сумма вероятностей того, что пользователь увидит рекоммендацию на то, что она окажется верной. Может применяться для любого списка, по которому скроллит (который просматривает) пользователь

WTA - Берется единица, если топовая рекомендация, имеющая самый большой предсказанный рейтинг из просмотренных пользователем, получила положительную оценку, иначе берется ноль. 

boughted = [1, 3, 5, 7, 9, 11], recommended = [2, 5, 7, 4, 11, 9, 8, 10, 12, 3]  
Посчитайте на этих данных pr@8, rec@8, AP@8, NDCG@8, RR@8, ERR@8

In [19]:
bought = 1, 3, 5, 7, 9, 11
recommended = 2, 5, 7, 4, 11, 9, 8, 10, 12, 3

def precision_at_k(recommended_list, bought_list, k=8):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    flags = np.isin(bought_list, recommended_list)
    precision = flags.sum() / len(recommended_list)
    
    return precision

def recall_at_k(recommended_list, bought_list, k=8):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    flags = np.isin(bought_list, recommended_list)
    recall = flags.sum() / len(bought_list)
    
    return recall
  
def avg_precision_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    flags = np.isin(recommended_list, bought_list)
    
    if sum(flags) == 0:
        return 0
    
    total_sum = 0
    for i in range(1, k+1): 
        if flags[i] == True:
            p_k = precision_at_k(recommended_list, bought_list, k=i)
            total_sum += p_k
            
    result = total_sum / sum(flags)
    
    return result

# Normalized discounted cumulative gain
def ndcg_at_k(recommended_list, bought_list, k=8): 
    recommended_list = np.array(recommended_list)
    kDCG_k = bought_list.count(recommended_list[0])
    
    t = 1
    for i in recommended_list[1:k]:
        t += 1
        n = bought_list.count(i)
        kDCG_k += n / np.log10(t)

    t = 1
    kiDCG_k = 1

    for i in range(1,k):
        t += 1
        kiDCG_k = kiDCG_k + 1 / np.log10(t)


    return kDCG_k / kiDCG_k

def reciprocal_rank_at_k(recommended_list, bought_list, k=8):
    n=0
    for i in recommended_list[:k]:
        n += bought_list.count(i)

    return n/k


# ERR@8
def err_k(recommended_list, bought_list, k=8):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list[:k])
    relevant_items_list = [(recommended_list[i] in bought_list) for i in range(recommended_list.shape[0])]
    relevant_item_rank = [i+1 for i in range(len(relevant_items_list)) if relevant_items_list[i]]    
    
    p_k = np.zeros(k)    
    for i in range(k):
        p_k[i] = (2**relevant_items_list[i] - 1) / (2**1 - 1)   
   
    P_k = np.zeros(k)    
    P_k[0] = p_k[0]
    for i in range(1, k):
        P_k[i] = p_k[i]
        for j in range(1, i):
            P_k[i] *= (1 - p_k[j]) 
        P_k[i] = P_k[i] / k
   
    return sum(P_k)

In [20]:
test_cases = [
    ["PR@8:", precision_at_k],  
    ["REC@8:", recall_at_k], 
    ["AP@8:", avg_precision_at_k], 
    ["NDCG@8:", ndcg_at_k], 
    ["RR@8 :", reciprocal_rank_at_k], 
    ["ERR@8:", err_k],
]

for test_case in test_cases:
    print(test_case[0], str(test_case[1](recommended, boughted, 8)))


PR@8: 0.5
REC@8: 0.6666666666666666
AP@8: 0.32
NDCG@8: 0.6215894130709897
RR@8 : 0.5
ERR@8: 0.125


$$bought = [1, 3, 5, 7,  9, 11]$$
$$recommended = [2, 5, 7, 4, 11, 9, 8, 10, 12, 3]$$
Для удобного среза брал с конца
$$precision = \frac{1 + 0 + 1 + 1 + 1 + 0}{8} = \frac{4}{8} = 0.5$$

$$recall = \frac{1 + 0 + 0 + 1 + 1 + 1 }{6} = \frac{4}{6} = 0.66$$
